In [3]:
# 04_nsga2_global_optimization.py
# Global NSGA-II optimization (saved-results + model artifacts)
# Author: Assistant (adapted for Apoorva's project)
# Run: python 04_nsga2_global_optimization.py

import json
import numpy as np
import pandas as pd
from pathlib import Path
import joblib
import matplotlib.pyplot as plt
import os
import sys
import traceback

# pymoo imports
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize

# sklearn for scaling
from sklearn.preprocessing import MinMaxScaler

# ----------------------------
# 0️⃣ Paths & config
# ----------------------------
BASE_DIR = Path("D:/Complete_Data/ml_part_nutrition_project")
PROCESSED_DIR = BASE_DIR / "processed_data"
RESULTS_DIR = BASE_DIR / "results"
MODELS_DIR = BASE_DIR / "models"

for d in (PROCESSED_DIR, RESULTS_DIR, MODELS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Possible filenames to try (user has used various names)
CANDIDATE_RECIPE_FILES = [
    PROCESSED_DIR / "recipes_final_for_optimization.csv",
    PROCESSED_DIR / "recipes_enriched.csv",
    PROCESSED_DIR / "recipes_with_env_metrics.csv",
    PROCESSED_DIR / "recipes_master.csv",
    PROCESSED_DIR / "recipes_safe.csv",
    PROCESSED_DIR / "recipes_safety_filtered.csv",
    PROCESSED_DIR / "recipes_enriched.csv"
]

OUTPUT_PLANS_CSV = RESULTS_DIR / "optimized_meal_plans_global.csv"
OUTPUT_PARETO_NPY = RESULTS_DIR / "pareto_F.npy"
OUTPUT_SOLUTIONS_NPY = RESULTS_DIR / "pareto_X.npy"
OUTPUT_CONFIG_JSON = RESULTS_DIR / "nsga2_run_config.json"
OUTPUT_PLOT = RESULTS_DIR / "pareto_plot.png"
SCALER_PATH = MODELS_DIR / "nsga2_scaler.joblib"

# Optimization hyperparams (tune as needed)
N_MEALS = 3            # number of recipes per meal plan
POP_SIZE = 80
N_GENERATIONS = 60
RANDOM_SEED = 42

# ----------------------------
# 1️⃣ Load recipes (robust)
# ----------------------------
def load_recipes():
    for p in CANDIDATE_RECIPE_FILES:
        if p.exists():
            try:
                df = pd.read_csv(p)
                print(f"Loaded recipes from: {p} -> shape: {df.shape}")
                return df, p
            except Exception as e:
                print(f"Failed to read {p}: {e}")
    raise FileNotFoundError("No recipes CSV found in processed_data. Check paths or run preprocessing first.")

try:
    recipes, loaded_from = load_recipes()
except Exception as e:
    print("ERROR: Could not load recipes. Aborting.")
    raise

# ----------------------------
# 2️⃣ Identify numeric features for optimization
#    We aim to include: energy_kcal, protein_g, fat_g, carbs_g, price, Total_emissions, Land_use_change
# ----------------------------
expected_features = [
    "energy_kcal_mean", "protein_g_mean", "fat_g_mean", "carbs_g_mean",
    "price_mean", "Total_emissions", "Total_emissions_mean", "Land use change", "Land_use_change", "Land use change_mean", "price"
]

# Normalize column names to help variance across files
recipes.columns = [c.strip() for c in recipes.columns]

# Build candidate list present in the df
present = {c: c for c in recipes.columns}
available_features = []
# heuristics: check multiple name variants
variants_mapping = {
    "energy_kcal_mean": ["energy_kcal_mean", "energy_kcal", "energy_kcal_mean_x", "energy_kcal_mean_y"],
    "protein_g_mean": ["protein_g_mean", "protein_g", "protein_g_mean_x", "protein_g_mean_y"],
    "fat_g_mean": ["fat_g_mean", "fat_g", "fat_g_mean_x", "fat_g_mean_y"],
    "carbs_g_mean": ["carbs_g_mean", "carbs_g", "carbs_g_mean_x", "carbs_g_mean_y"],
    "price_mean": ["price_mean", "price", "usdprice", "price_x", "price_y"],
    "Total_emissions": ["Total_emissions", "Total_emissions_mean", "Total_emissions_mean_x", "Total_emissions_mean_y"],
    "Land_use_change": ["Land use change", "Land_use_change", "Land use change_mean"]
}

for canonical, variants in variants_mapping.items():
    for v in variants:
        if v in recipes.columns:
            available_features.append(v)
            break

# If we still have too few features, try any numeric columns
if len(available_features) < 3:
    numeric_cols = recipes.select_dtypes(include=[np.number]).columns.tolist()
    # exclude obvious ID-like columns (fdc etc)
    numeric_cols = [c for c in numeric_cols if "id" not in c.lower()]
    # take up to 6 numeric columns
    available_features = numeric_cols[:6]

print("Available numeric features chosen for optimization:", available_features)

# ----------------------------
# 3️⃣ Basic validation & fallback
# ----------------------------
if recipes.shape[0] == 0:
    raise ValueError("Recipes dataset is empty. Please run preprocessing (environment mapping / nutrient mapping) first.")

if len(available_features) == 0:
    raise ValueError("No usable numeric features found in recipes. Inspect processed CSV columns.")

# Build a working dataframe with required columns
work = recipes.copy()
# ensure title exists
if "recipe_title" not in work.columns:
    # try 'title' fallback
    if "title" in work.columns:
        work = work.rename(columns={"title": "recipe_title"})
    else:
        # create a synthetic title from index
        work["recipe_title"] = work.index.astype(str)
        print("No title column found — synthetic titles added.")

# Ensure feature cols exist (fill na with median)
for col in available_features:
    if col not in work.columns:
        work[col] = np.nan
work[available_features] = work[available_features].apply(pd.to_numeric, errors="coerce")
work[available_features] = work[available_features].fillna(work[available_features].median())

n_recipes = len(work)
print(f"Working recipe count: {n_recipes}. Using {len(available_features)} features.")

if n_recipes < N_MEALS:
    raise ValueError(f"Not enough recipes ({n_recipes}) to build meal plans of {N_MEALS} recipes each.")

# ----------------------------
# 4️⃣ Scale features and save scaler
# ----------------------------
scaler = MinMaxScaler()
X = work[available_features].values
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, SCALER_PATH)
print(f"Saved scaler to: {SCALER_PATH}")

# attach scaled columns to dataframe (optional)
scaled_cols = [f"scaled_{i}" for i in range(X_scaled.shape[1])]
for i, c in enumerate(scaled_cols):
    work[c] = X_scaled[:, i]

# ----------------------------
# 5️⃣ Define NSGA-II problem (integer indices)
# ----------------------------
class MealPlanningProblem(ElementwiseProblem):
    def __init__(self, n_meals, data_len):
        # lower and upper bounds must be arrays for n_var
        xl = np.zeros(n_meals, dtype=int)
        xu = np.full(n_meals, data_len - 1, dtype=int)
        super().__init__(n_var=n_meals, n_obj=3, xl=xl, xu=xu, elementwise_evaluation=True)
        self.data = work
        self.features = available_features

    def _evaluate(self, x, out, *args, **kwargs):
        # x is array of indices (floats) -> cast to int
        idx = np.array(x).astype(int)
        sel = self.data.iloc[idx]
        # Objectives: minimize emissions & cost, maximize protein (so return -protein to minimize neg)
        # Choose columns robustly:
        # emissions
        emis_col_candidates = [c for c in self.features if "emiss" in c.lower() or "total_emission" in c.lower() or "total_emissions" in c.lower()]
        cost_col_candidates = [c for c in self.features if "price" in c.lower() or "cost" in c.lower() or "usdprice" in c.lower()]
        prot_col_candidates = [c for c in self.features if "protein" in c.lower()]
        # fallback picks
        emis_col = emis_col_candidates[0] if emis_col_candidates else (self.features[-1])
        cost_col = cost_col_candidates[0] if cost_col_candidates else (self.features[-2] if len(self.features) >= 2 else self.features[-1])
        prot_col = prot_col_candidates[0] if prot_col_candidates else (self.features[1] if len(self.features) > 1 else self.features[0])

        total_emission = float(sel[emis_col].sum()) if emis_col in sel.columns else 0.0
        total_cost = float(sel[cost_col].sum()) if cost_col in sel.columns else 0.0
        total_protein = float(sel[prot_col].sum()) if prot_col in sel.columns else 0.0
        # energy penalty to keep meal plans reasonable if energy exists
        energy_cols = [c for c in self.features if "energy" in c.lower() or "kcal" in c.lower()]
        energy_penalty = 0.0
        if energy_cols:
            total_cal = float(sel[energy_cols[0]].sum())
            # aim for 2000 kcal per day (simple)
            energy_penalty = abs(total_cal - 2000) / 2000.0

        # Compose objectives: minimize emission, cost, negative protein (so maximize protein)
        f1 = total_emission + energy_penalty  # minimize emission (with energy penalty)
        f2 = total_cost
        f3 = -total_protein  # maximize protein -> minimize negative

        out["F"] = [f1, f2, f3]

# ----------------------------
# 6️⃣ Run optimizer
# ----------------------------
np.random.seed(RANDOM_SEED)

problem = MealPlanningProblem(n_meals=N_MEALS, data_len=n_recipes)
algorithm = NSGA2(pop_size=POP_SIZE)

termination = get_termination("n_gen", N_GENERATIONS)

print(f"Running NSGA-II: pop_size={POP_SIZE}, n_gen={N_GENERATIONS}, n_meals={N_MEALS}")
res = None
try:
    res = minimize(problem, algorithm, termination, verbose=True)
except Exception as e:
    print("Optimization failed with an exception:")
    traceback.print_exc()
    sys.exit(1)

print("Optimization completed.")

# ----------------------------
# 7️⃣ Extract & save results
# ----------------------------
F = np.array(res.F)
X = np.array(res.X)

np.save(OUTPUT_PARETO_NPY, F)
np.save(OUTPUT_SOLUTIONS_NPY, X)
print(f"Saved pareto arrays: {OUTPUT_PARETO_NPY}, {OUTPUT_SOLUTIONS_NPY}")

# Build results DataFrame: one row per solution
plans = []
for i, sol in enumerate(X):
    idx = np.array(sol).astype(int)
    sel = work.iloc[idx]
    plan = {
        "plan_id": i + 1,
        "recipe_indices": idx.tolist(),
        "recipes": " | ".join(sel["recipe_title"].astype(str).tolist()),
        "total_emission": float(sel[[c for c in work.columns if "emiss" in c.lower() or "Total_emissions" in c]].sum().sum()) if any("emiss" in c.lower() for c in work.columns) else float(np.nan),
        "total_cost": float(sel[[c for c in work.columns if "price" in c.lower() or "usdprice" in c.lower()]].sum().sum()) if any("price" in c.lower() for c in work.columns) else float(np.nan),
        "total_protein": float(sel[[c for c in work.columns if "protein" in c.lower()]].sum().sum()) if any("protein" in c.lower() for c in work.columns) else float(np.nan)
    }
    plans.append(plan)

plans_df = pd.DataFrame(plans)
plans_df.to_csv(OUTPUT_PLANS_CSV, index=False)
print(f"Saved optimized meal plans to: {OUTPUT_PLANS_CSV}")
print("Sample plans:")
print(plans_df.head())

# ----------------------------
# 8️⃣ Save run config & metadata
# ----------------------------
config = {
    "base_dir": str(BASE_DIR),
    "loaded_from": str(loaded_from),
    "n_recipes": int(n_recipes),
    "n_meals": N_MEALS,
    "pop_size": POP_SIZE,
    "n_generations": N_GENERATIONS,
    "random_seed": RANDOM_SEED,
    "features_used": available_features,
    "plans_csv": str(OUTPUT_PLANS_CSV),
    "pareto_F": str(OUTPUT_PARETO_NPY),
    "pareto_X": str(OUTPUT_SOLUTIONS_NPY),
    "scaler": str(SCALER_PATH)
}
with open(OUTPUT_CONFIG_JSON, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)
print(f"Saved run config to: {OUTPUT_CONFIG_JSON}")

# ----------------------------
# 9️⃣ Pareto visualization (2D or 3D)
# ----------------------------
try:
    F = F if isinstance(F, np.ndarray) else np.array(F)
    # We used 3 objectives (f1: emission, f2: cost, f3: -protein)
    if F.shape[1] >= 3:
        # plot emission vs cost colored by protein
        fig = plt.figure(figsize=(8, 6))
        sc = plt.scatter(F[:, 0], F[:, 1], c=-F[:, 2], s=40)
        plt.xlabel("Objective 1 (Emission + energy penalty) — lower better")
        plt.ylabel("Objective 2 (Cost) — lower better")
        cbar = plt.colorbar(sc)
        cbar.set_label("Protein (higher better)")
        plt.title("Pareto front (emission vs cost; protein shown by color)")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(OUTPUT_PLOT, dpi=250)
        print(f"Saved Pareto plot to: {OUTPUT_PLOT}")
        plt.close()
    else:
        # fallback 2D
        plt.figure(figsize=(8,6))
        plt.scatter(F[:,0], F[:,1])
        plt.xlabel("Obj1"); plt.ylabel("Obj2")
        plt.title("Pareto front (2D)")
        plt.grid(True)
        plt.savefig(OUTPUT_PLOT, dpi=250)
        plt.close()
        print(f"Saved Pareto plot to: {OUTPUT_PLOT}")
except Exception:
    print("Failed to create Pareto visualization:")
    traceback.print_exc()

print("NSGA-II global optimization run finished successfully.")


Loaded recipes from: D:\Complete_Data\ml_part_nutrition_project\processed_data\recipes_final_for_optimization.csv -> shape: (20130, 11)
Available numeric features chosen for optimization: ['energy_kcal_mean', 'protein_g_mean', 'fat_g_mean', 'carbs_g_mean', 'price_mean', 'Total_emissions_mean']
Working recipe count: 20130. Using 6 features.
Saved scaler to: D:\Complete_Data\ml_part_nutrition_project\models\nsga2_scaler.joblib
Running NSGA-II: pop_size=80, n_gen=60, n_meals=3
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       80 |      2 |             - |             -
     2 |      160 |      3 |  0.2239186235 |             f
     3 |      240 |      6 |  0.0660245083 |         ideal
     4 |      320 |      7 |  0.1013811645 |         ideal
     5 |      400 |      8 |  0.0227681898 |             f
     6 |      480 |      8 |  0.000000E+00 |             f
     7 |      560 |     10 |  0.1319985037 |         ideal
     8 |      640 |     11 |  0.0125133370 |     

In [2]:
import pandas as pd
from pathlib import Path

base = Path("D:/Complete_Data/ml_part_nutrition_project/processed_data")

for name in [
    "recipes_with_env_metrics.csv",
    "recipes_enriched.csv",
    "recipes_final_for_optimization.csv",
    "recipes_safety_filtered.csv"
]:
    path = base / name
    if path.exists():
        df = pd.read_csv(path)
        print(f"\n{name}: shape={df.shape}")
        print("Columns:", list(df.columns))
        print(df.head(2))



recipes_with_env_metrics.csv: shape=(20130, 3)
Columns: ['recipe_title', 'Total_emissions', 'Land_use_change']
   recipe_title  Total_emissions  Land_use_change
0             0         4.133333        -0.026667
1             1         3.996429         0.007143

recipes_enriched.csv: shape=(20130, 9)
Columns: ['recipe_id', 'recipe_title', 'ingredient_text', 'energy_kcal_mean', 'protein_g_mean', 'fat_g_mean', 'carbs_g_mean', 'price_mean', 'Total_emissions_mean']
   recipe_id                                 recipe_title  \
0        NaN              Lentil, Apple, and Turkey Wrap    
1        NaN  Boudin Blanc Terrine with Red Onion Confit    

                                     ingredient_text  energy_kcal_mean  \
0  4 cups low sodium vegetable or chicken stock, ...               NaN   
1  1 1 2 cups whipping cream, 2 medium onions cho...               NaN   

   protein_g_mean  fat_g_mean  carbs_g_mean  price_mean  Total_emissions_mean  
0             NaN         NaN           NaN    